## About
This notebook is used to compute the best aligned rosetta pose from TM output.<br/>
This pose can later be used for other analyses

**Note:** Some parts of this code may be sensitive to the RNA chosen. These regions are marked out with a <i>'#TODO: Change accordingly'</i> tag

In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
import os
%run ../PyRosetta/General.ipynb

/home/venkata/micromamba/thermomaps/lib/python3.10/site-packages/Bio/Application/__init__.py:40: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│              See LICENSE.md or email license@uw.edu for details              │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.Release.python310.linux 2024.08+release.717d2e8232174371f0c672564f23a097062db88a 2024-02-21T10:16:44] retrieved from: http://www.pyrosetta.org


In [5]:
ROOT_DIR="/home/venkata/python/PyRosetta/R1107/server/example/" #TODO: Change Accordingly
print("Rosetta folder found:",("rosetta" in os.listdir(ROOT_DIR)))

Rosetta folder found: True


In [8]:
in_file=ROOT_DIR+"/rosetta/combined.out"
ref_pose=LoadedPDB(ROOT_DIR+"../../7qr4_clean_noprotein.pdb") #TODO: Change Accordingly (Set it to the path of your crystal PDB)
poses=[pose for pose in poses_from_silent(in_file)]
print(len(poses),"rosetta structures loaded.")

500 rosetta structures loaded.


### Sanity check
Making sure that all the hydrogen and non-hydrogen atoms align (i.e. all atoms correspond 1-1)

In [9]:
for rid,resid in enumerate(ref_pose):
    for i,atom in enumerate(resid.atoms()):
        chk1=resid.atom_is_hydrogen(i+1)
        chk2=poses[0].residue(rid+1).atom_is_hydrogen(i+1)
        if chk1!=chk2:
            print("ERR:\t",end="")
            print(rid,i)
else:
    print("All good")

All good


In [10]:
resolved_rmsds=[]
for i in range(len(poses)):
    print("Rosetta Structure:",i)
    aligned_pose=align_poses(ref_pose,poses[i],silent=True)
    print("\tPrevious RMSD:",get_rmsd(ref_pose,poses[i]))
    final_rmsd=get_rmsd(ref_pose,aligned_pose)
    print("\tFinal RMSD:",final_rmsd)
    #aligned_pose.dump_pdb("R1107/server/R1107_long_rosetta_"+str(i)+"_relaxed_aligned_noH.pdb") # TODO: Change accordingly (Uncomment to dump an aligned PDB file)
    resolved_rmsds.append(final_rmsd)
resolved_rmsds=np.array(resolved_rmsds)

Rosetta Structure: 0
	Previous RMSD: 43.914457170261
	Final RMSD: 18.889876344990036
Rosetta Structure: 1
	Previous RMSD: 41.10033153189976
	Final RMSD: 23.315215100912127
Rosetta Structure: 2
	Previous RMSD: 48.0207661898853
	Final RMSD: 22.84878656728571
Rosetta Structure: 3
	Previous RMSD: 58.51399301433253
	Final RMSD: 19.91027928591436
Rosetta Structure: 4
	Previous RMSD: 43.4522495156865
	Final RMSD: 15.76750740225815
Rosetta Structure: 5
	Previous RMSD: 34.580178886591256
	Final RMSD: 24.879072879288927
Rosetta Structure: 6
	Previous RMSD: 56.839158113874554
	Final RMSD: 25.54276360972105
Rosetta Structure: 7
	Previous RMSD: 45.50343754545681
	Final RMSD: 22.88804566325193
Rosetta Structure: 8
	Previous RMSD: 37.425735134315325
	Final RMSD: 22.97690137414751
Rosetta Structure: 9
	Previous RMSD: 48.99260193677845
	Final RMSD: 27.00536904301768
Rosetta Structure: 10
	Previous RMSD: 43.14762349545059
	Final RMSD: 26.01355888714438
Rosetta Structure: 11
	Previous RMSD: 45.7660045989

In [11]:
# Save the RMSDs
np.save(ROOT_DIR+"/rosetta/RMSD_crystal.npy",resolved_rmsds)

# Print out the lowest RMSD
print("Lowest RMSD:",np.min(resolved_rmsds))

#### Save the pose closest to the crystal structure

In [12]:
best_rosetta=poses[np.argmin(resolved_rmsds)]

# Align the closest match to the crystal structure and save it
aligned_best_rosetta=align_poses(ref_pose,best_rosetta,silent=True,max_iter=5000)
aligned_best_rosetta.dump_pdb(ROOT_DIR+"/rosetta/crystal_closest.pdb")

True